In [1]:
from traitlets.config.manager import BaseJSONConfigManager
path = "/home/nikorose/Workspace/anaconda3/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
              'theme': 'simple',
              'transition': 'zoom',
              'start_slideshow_at': 'selected',
              'scroll': True,
})

{'scroll': True,
 'start_slideshow_at': 'selected',
 'theme': 'simple',
 'transition': 'zoom'}

# Algorithm to obtain quasi-stiffness of natural and impaired ankle joint
<div style="text-align:center" markdown="1">
<div>
Presented by: Edwin Nikolay Prieto Parrado M.Sc.
<div>
Mechanics and Mechatronics Doctorate programme
<div>
PhD Student
<center><img src="files/figures/GIBM.png" alt="Drawing" style="width: 300px;"/></center>
</div>


According to the first objective of the thesis, which is: 
"Identify biomechanics parameters and the work-loop of ankle joint in transtibial amputees and able-bodied patients with the aim of determining energetics requirements".

In these order of ideas, We checked literature about gait analysis of impaired and able-bodied patients at different walking speeds, as it is shown below:

<center><img src="files/figures/MindMapObjective1.png" alt="Drawing" style="width: 1000px;"/></center>


Hence, We begun to explore the data in the same order.

## A multiple-task gait analysis approach: Kinematic, kinetic and EMG reference data for healthy young and adult subjects by (Bovi et al., 2011)

In [1]:
# %load FerrarinData.py
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
from bokeh.plotting import figure, output_file, show, output_notebook
import warnings
import numpy as np
from bokeh.layouts import row, column
from bokeh.palettes import Spectral6

# Trying to load bokeh inline
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE)
from scipy.integrate import simps
from sklearn.svm    import SVR
from sklearn import linear_model
warnings.filterwarnings('ignore')
#output_notebook()
##Cleaning and selecting the data

localdir =  "/home/nikorose/Dropbox/Gait_Analysis_data/Downloaded/Ferrarin"
FerrarinData =  pd.ExcelFile(localdir+'/mmc3.xls')

#Setting Angles
Angles = FerrarinData.parse("Joint Rotations")
Angles = Angles[Angles.columns[3:55:3]]
Angles.columns = ['Natural_y','XSy','Sy','My','Ly','Toe_y','Heel_y','Ascending_y','Descending_y','Natural_a','XSa','Sa','Ma','La','Toe_a','Heel_a','Ascending_a','Descending_a']
AnkleAngles = Angles.loc[708:807]


#Setting Moments
Moments = FerrarinData.parse("Joint Moments")
Moments = Moments[Moments.columns[3:55:3]]
Moments.columns = ['Natural_y','XSy','Sy','My','Ly','Toe_y','Heel_y','Ascending_y','Descending_y','Natural_a','XSa','Sa','Ma','La','Toe_a','Heel_a','Ascending_a','Descending_a']
AnkleMoment = Moments.loc[404:503]

#Defining Gait cycle array
cycle = pd.DataFrame({"cycle %":np.linspace(0,10,100)})

AnkleAngles.head()

Loading BokehJS ...

Natural_y        XSy         Sy         My         Ly      Toe_y  \
708 -22.475384 -26.111954 -24.006180 -21.000801 -20.627172 -44.634789   
709 -23.081146 -26.707436 -24.720978 -21.572119 -21.362429 -42.627811   
710 -23.673306 -27.316370 -25.412529 -22.235031 -21.960266 -40.437008   
711 -24.063673 -27.802595 -25.866137 -22.689116 -22.370613 -38.323177   
712 -24.177179 -28.042812 -26.041065 -22.862928 -22.436058 -36.376915   

        Heel_y  Ascending_y  Descending_y  Natural_a        XSa         Sa  \
708  -8.910422   -13.743444    -41.575375 -22.836117 -23.786188 -23.295578   
709  -9.419728   -12.744245    -37.979549 -23.943291 -25.006784 -24.171341   
710  -9.956508   -11.695560    -34.199963 -25.236042 -26.335239 -25.189970   
711 -10.478696   -10.718312    -30.361139 -26.167620 -27.118679 -25.935953   
712 -10.941799    -9.812226    -26.556541 -26.457401 -27.189493 -26.228153   

            Ma         La      Toe_a     Heel_a  Ascending_a  Descending_a  
708 -21.777031 -21.707331 -46.698643  -8.201367   -13.944626    -42.540791  
709 -22.972391 -23.295666 -43.766171  -8.740160   -12.539468    -38.905220  
710 -24.479002 -24.849926 -40.857220  -9.378276   -11.249466    -34.897137  
711 -25.681875 -25.780676 -38.408676  -9.989401   -10.128537    -31.041113  
712 -26.128460 -25.840458 -36.396976 -10.452395    -9.044915    -27.432087

**Table 1** Kinematic analysis of different models taken by Bovi et al,. 2011 <cite data-cite="Bovi2011"></cite>

In [3]:
AnkleMoment.head()

Natural_y       XSy        Sy        My        Ly     Toe_y    Heel_y  \
404  -0.053062 -0.039622 -0.048601 -0.050550 -0.067037  0.256900 -0.061094   
405  -0.059390 -0.048647 -0.057349 -0.055973 -0.071651  0.321804 -0.072053   
406  -0.064825 -0.056743 -0.064983 -0.060446 -0.075764  0.390163 -0.083285   
407  -0.067765 -0.063204 -0.070856 -0.062345 -0.077679  0.463005 -0.094618   
408  -0.068262 -0.067089 -0.073421 -0.061798 -0.076813  0.536315 -0.105816   

     Ascending_y  Descending_y  Natural_a       XSa        Sa        Ma  \
404     0.049866      0.236713  -0.036952 -0.025355 -0.035409 -0.044590   
405     0.067745      0.314066  -0.043153 -0.031829 -0.041510 -0.051203   
406     0.087312      0.392670  -0.048079 -0.037504 -0.046525 -0.056288   
407     0.107985      0.473328  -0.050928 -0.041363 -0.049730 -0.059032   
408     0.129245      0.552243  -0.050595 -0.042443 -0.049887 -0.058549   

           La     Toe_a    Heel_a  Ascending_a  Descending_a  
404 -0.077897  0.239192 -0.059136     0.023803      0.177180  
405 -0.086495  0.308561 -0.072221     0.037531      0.250662  
406 -0.093423  0.385212 -0.085959     0.054205      0.333815  
407 -0.097959  0.468567 -0.100018     0.073555      0.423818  
408 -0.097810  0.555950 -0.113811     0.095251      0.516390

**Table 2** Inverse dynamics analysis of different models taken by Bovi et al,. 2011 <cite data-cite="Bovi2011"></cite

In [4]:

cuadratura = 700
#Figure for young people
f = figure(x_axis_label='Right Ankle Angle (Deg)', y_axis_label='Moment (Nm/kg)', plot_width=cuadratura, plot_height=cuadratura, title ="Ankle Quasi-Stiffness in different young people")
fa = figure(x_axis_label='Right Ankle Angle (Deg)', y_axis_label='Moment (Nm/kg)', plot_width=cuadratura, plot_height=cuadratura, title ="Ankle Quasi-Stiffness in different young people with different walking modes")
color = 0
area= []

## Defining some regression techniques 
svr_rbf = SVR(kernel='rbf', C=1e2, gamma=0.1) # Configuring Support Vector Regression
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=3)
## Doing the regression of data
y_linear_lr = linear_model.LinearRegression().fit(AnkleAngles[:int(len(AnkleAngles)*1)], AnkleMoment[:int(len(AnkleAngles)*1)]).predict(AnkleAngles[:int(len(AnkleAngles)*1)])
y_linear_data = pd.DataFrame(y_linear_lr)
y_linear_data.columns = ['Natural_y','XSy','Sy','My','Ly','Toe_y','Heel_y','Ascending_y','Descending_y','Natural_a','XSa','Sa','Ma','La','Toe_a','Heel_a','Ascending_a','Descending_a']


for diff in Angles.columns[:5]:
    color = color + 1
    f.circle(x = AnkleAngles[diff],y=AnkleMoment[diff], color = Spectral6[color], size = cycle["cycle %"], legend = diff)
    f.line(x=AnkleAngles[diff], y=y_linear_data[diff], color=Spectral6[color], line_width=2, legend=diff)
    f.legend.location = "top_left"
    area.append(simps(AnkleMoment[diff].values, -AnkleAngles[diff].values, dx=1))

color = 0
for diff in Angles.columns[5:9]:
    color = color + 1
    fa.circle(x = AnkleAngles[diff],y=AnkleMoment[diff], color = Spectral6[color], size = cycle["cycle %"], legend = diff)
    fa.line(x=AnkleAngles[diff], y=y_linear_data[diff], color=Spectral6[color], line_width=2, legend=diff)
    fa.legend.location = "top_left"
    area.append(simps(AnkleMoment[diff].values, -AnkleAngles[diff].values, dx=1))

show(row(f,fa), notebook_handle=True)



In [5]:

#Figure for adult people - normal walking
fb = figure(x_axis_label='Right Ankle Angle (Deg)', y_axis_label='Moment (Nm/kg)', plot_width=cuadratura, plot_height=cuadratura, title ="Ankle Quasi-Stiffness in different Adult people")
color = 0
for diff in Angles.columns[9:14]:
    color = color + 1
    fb.circle(x = AnkleAngles[diff],y = AnkleMoment[diff], color = Spectral6[color], size = cycle["cycle %"], legend = diff)
    fb.line(x=AnkleAngles[diff], y=y_linear_data[diff], color=Spectral6[color], line_width=2, legend=diff)
    fb.legend.location = "top_left"
    area.append(simps(AnkleMoment[diff].values, -AnkleAngles[diff].values, dx=0.5))

#Figure for adult people - Anormal walking
fc = figure(x_axis_label='Right Ankle Angle (Deg)', y_axis_label='Moment (Nm/kg)', plot_width=cuadratura, plot_height=cuadratura, title ="Ankle Quasi-Stiffness in different Adult people")
color = 0
for diff in Angles.columns[14:]:
    color = color + 1
    fc.circle(x = AnkleAngles[diff],y = AnkleMoment[diff], color = Spectral6[color], size = cycle["cycle %"], legend = diff)
    fc.line(x=AnkleAngles[diff], y=y_linear_data[diff], color=Spectral6[color], line_width=2, legend=diff)
    fc.legend.location = "top_left"
    area.append(simps(AnkleMoment[diff].values, -AnkleAngles[diff].values, dx=0.5))
    
show(row(fb,fc), notebook_handle=True)


In [6]:
# Printing the Area of the loop, Which is equal to propulsion work
print ('Propulsion work obtained for young and adult population by Simpson integration:')
area_to_print = pd.DataFrame ([area[:9],area[9:]], index=["Young","Adult"], columns=['Natural','XS','S','M','L','Toe','Heel','Ascending','Descending'])
area_to_print


Propulsion work obtained for young and adult population by Simpson integration:


Natural         XS          S          M          L       Toe  \
Young  17.066538  10.614100  15.289586  16.204318  19.197126  4.689266   
Adult  17.766885  12.800099  16.312120  20.620187  26.919197  5.916090   

           Heel  Ascending  Descending  
Young  0.355437  22.869532  -17.954274  
Adult  0.514594  23.057421  -19.292681

In [7]:
y_linear_lr2 = linear_model.LinearRegression()
y_linear_lr2.fit(AnkleAngles[:int(len(AnkleAngles)*1)], AnkleMoment[:int(len(AnkleAngles)*1)])
y_linear_lr2.predict(AnkleAngles[:int(len(AnkleAngles)*1)])
# coefficients of error
# The mean squared error
# Explained variance score: 1 is perfect prediction

stats = pd.DataFrame([len(y_linear_lr2.coef_[0]),np.mean((y_linear_lr - AnkleMoment.values) ** 2),
                    y_linear_lr2.score(AnkleAngles[int(len(AnkleAngles)*0.7):], 
                    AnkleMoment[int(len(AnkleAngles)*0.7):])],
                    index = ["Coefficients size", "Mean Squared error", "Variance Score"], columns = ["Results"])
stats

Results
Coefficients size   18.000000
Mean Squared error   0.000179
Variance Score      -1.399547

# Validating and comparing data 
<div style="width:500 px" alt="Drawing">
<div style="float:left; width:100 px"><img src="files/figures/fastQuasi.png"/></div>
<div style="float:left; width:100 px"><img src="files/figures/NormalQuasi.png"/></div>
<div style="float:left; width:100 px"><img src="files/figures/SlowQuasi.png"/></div>

<div style="clear:both"></div>
</div>


**Fig. 3**Ankle moment versus ankle angle plots for a typical subject walking at three speeds (slow, normal, and fast). Walking speeds for the three trials were 1.2 m/s (slow), 1.5 m/s (normal), and 1.9 m/s (fast). Distances between dots on the graphs indicate 1% of the stance time. The white-filled squares indicate the opposite heel contact event. The hysteresis loopfor the slow walking speed moves in a clockwise rotation while the loopfor fast walking speed moves in a counter- clockwise rotation.

# Validating and comparing results
<div style="width:100 px" alt="Drawing">
<div style="float:left; width:50 px"><img src="files/figures/Shamaei.png"/></div>
<div style="clear:both"></div>
</div>

**Fig. 4** Ankle moment vs. relative angle curve for a representative subject walking at 1.75 m/s. Taken from (Shamaei et al., 2013)



<div style="text-align:center" markdown="1">
<div>
<font size="10">
Thank you
<div></font>
<div>
<div>
<font size="6">
enprietop@unal.edu.co
<div>
<div>
<div>
PhD Student
<center><img src="files/figures/GIBM.png" alt="Drawing" style="width: 200px;"/></center>
</div>
</font>

# References

